In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
# Transformasi Dataset
# Transformasi dataset dengan langkah-langkah berikut:
transform = transforms.Compose([
    # Mengubah gambar menjadi format tensor (mengubah gambar menjadi array multidimensi PyTorch)
    transforms.ToTensor(),
    # Normalisasi nilai piksel ke rentang [-1, 1] dengan mean 0.5 dan standar deviasi 0.5
    transforms.Normalize((0.5,), (0.5,))
])

In [3]:
# Load Fashion MNIST Dataset
batch_size = 32
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

100%|██████████| 26.4M/26.4M [00:01<00:00, 15.9MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 272kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 5.04MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 11.9MB/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [4]:
# Membagi dataset pelatihan menjadi subset pelatihan dan validasi
train_size = int(0.8 * len(train_dataset))  # Mengambil 80% data untuk pelatihan
val_size = len(train_dataset) - train_size  # Sisanya (20%) digunakan untuk validasi
train_data, val_data = random_split(train_dataset, [train_size, val_size])  # Membagi dataset pelatihan

# Membuat DataLoader untuk subset pelatihan, validasi, dan pengujian
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)  # Loader untuk data pelatihan
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)     # Loader untuk data validasi
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # Loader untuk data pengujian

In [5]:
# Definisi Arsitektur CNN
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling_type='max'):
        super(CNN, self).__init__()
        # Padding untuk mempertahankan ukuran spatial pada output convolusi
        padding = kernel_size // 2

        # Lapisan convolusi pertama (input: 1 channel, output: 32 channel)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=padding)  # Input grayscale (1 channel)

        # Lapisan convolusi kedua (input: 32 channel, output: 64 channel)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=padding)

        # Pooling layer, dapat berupa MaxPooling atau AveragePooling tergantung parameter
        if pooling_type == 'max':
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Mengambil nilai maksimum
        elif pooling_type == 'avg':
            self.pool = nn.AvgPool2d(kernel_size=2, stride=2)  # Mengambil nilai rata-rata

        # Fully connected layer pertama (menghubungkan feature map ke 128 unit)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Ukuran feature map setelah pooling adalah 7x7

        # Fully connected layer kedua (menghubungkan ke 10 output untuk 10 kelas)
        self.fc2 = nn.Linear(128, 10)

        # Fungsi aktivasi ReLU
        self.relu = nn.ReLU()

        # Dropout untuk mencegah overfitting
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Forward pass melalui lapisan convolusi pertama, ReLU, dan pooling
        x = self.pool(self.relu(self.conv1(x)))

        # Forward pass melalui lapisan convolusi kedua, ReLU, dan pooling
        x = self.pool(self.relu(self.conv2(x)))

        # Meratakan output menjadi vektor 1 dimensi
        x = x.view(-1, 64 * 7 * 7)

        # Forward pass melalui fully connected layer pertama dan ReLU
        x = self.relu(self.fc1(x))

        # Dropout untuk regularisasi
        x = self.dropout(x)

        # Forward pass melalui fully connected layer kedua untuk output prediksi
        x = self.fc2(x)

        return x

In [ ]:
# Training dan Evaluasi
def train_and_evaluate(kernel_size, pooling_type, optimizer_type, epochs, early_stopping_patience=10):
    # Tentukan perangkat (GPU jika tersedia, jika tidak gunakan CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Inisialisasi model dengan parameter kernel_size dan pooling_type
    model = CNN(kernel_size=kernel_size, pooling_type=pooling_type).to(device)

    # Loss function (CrossEntropyLoss untuk tugas klasifikasi multi-kelas)
    criterion = nn.CrossEntropyLoss()

    # Inisialisasi optimizer berdasarkan pilihan optimizer_type
    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif optimizer_type == 'RMSProp':
        optimizer = optim.RMSprop(model.parameters(), lr=0.01)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Scheduler untuk menyesuaikan learning rate secara dinamis
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=False)

    # Variabel untuk mencatat performa terbaik
    best_loss = float('inf')
    patience_counter = 0

    # Menampilkan konfigurasi eksperimen
    print(f"Evaluating with params: {{'epochs': {epochs}, 'kernel_size': {kernel_size}, 'optimizer_type': '{optimizer_type}', 'pooling_type': '{pooling_type}'}}")

    for epoch in range(epochs):
        # === Training Step ===
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            # Transfer data ke perangkat (GPU/CPU)
            images, labels = images.to(device), labels.to(device)

            # Reset gradien
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)

            # Hitung loss
            loss = criterion(outputs, labels)

            # Backward pass dan update parameter
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)

        # === Validation Step ===
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Update learning rate berdasarkan scheduler
        scheduler.step(val_loss)

        # Early Stopping jika performa tidak membaik
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print("Early stopping triggered.")
                break

    # === Evaluasi Akurasi pada Test Dataset ===
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # Prediksi label
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

# Eksperimen dengan Semua Kombinasi Parameter
kernel_sizes = [3, 5, 7]  # Ukuran kernel untuk lapisan convolusi
pooling_types = ['max', 'avg']  # Jenis pooling (MaxPooling atau AvgPooling)
optimizers = ['SGD', 'RMSProp', 'Adam']  # Optimizer yang akan diuji
epoch_list = [5, 50, 100, 250, 350]  # Jumlah epoch untuk pelatihan

results = []

# Kombinasi semua parameter
for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for optimizer in optimizers:
            for epochs in epoch_list:
                print(f"Running experiment with kernel_size={kernel_size}, pooling_type={pooling_type}, optimizer={optimizer}, epochs={epochs}")
                accuracy = train_and_evaluate(kernel_size, pooling_type, optimizer, epochs, early_stopping_patience=10)
                results.append((kernel_size, pooling_type, optimizer, epochs, accuracy))

# Menampilkan Hasil Akhir
print("\nHasil Akhir Eksperimen:")
for result in results:
    print(f"Kernel Size: {result[0]}, Pooling: {result[1]}, Optimizer: {result[2]}, Epochs: {result[3]}, Accuracy: {result[4]:.2f}%")

Running experiment with kernel_size=3, pooling_type=max, optimizer=SGD, epochs=5
Evaluating with params: {'epochs': 5, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/5, Loss: 0.6098, Val Loss: 0.3462
Epoch 2/5, Loss: 0.3773, Val Loss: 0.2912
Epoch 3/5, Loss: 0.3225, Val Loss: 0.2840
Epoch 4/5, Loss: 0.2890, Val Loss: 0.2626
Epoch 5/5, Loss: 0.2602, Val Loss: 0.2497
Accuracy: 90.43%
Running experiment with kernel_size=3, pooling_type=max, optimizer=SGD, epochs=50
Evaluating with params: {'epochs': 50, 'kernel_size': 3, 'optimizer_type': 'SGD', 'pooling_type': 'max'}
Epoch 1/50, Loss: 0.6123, Val Loss: 0.3467
Epoch 2/50, Loss: 0.3785, Val Loss: 0.2991
Epoch 3/50, Loss: 0.3252, Val Loss: 0.2670
Epoch 4/50, Loss: 0.2912, Val Loss: 0.2619
Epoch 5/50, Loss: 0.2636, Val Loss: 0.2399
Epoch 6/50, Loss: 0.2481, Val Loss: 0.2353
Epoch 7/50, Loss: 0.2297, Val Loss: 0.2343
Epoch 8/50, Loss: 0.2159, Val Loss: 0.2380
Epoch 9/50, Loss: 0.2015, Val Loss: 0.2326
Epoch 10/50, Loss: 0.1907, Val Loss: 0.2225
Epoch 11/50, Loss: 0.1796, Val Loss: 0.2292
Epoch 12/50, Loss: 0.1690, Val Loss: 0.2401
Epoch 13/50, Loss: 0.1593, Val Loss: 0.2261
Epoch 14/50, Loss: 0.15